In [1]:
import numpy as np
from astropy.io import fits


### Retrieving the relevant CCD Data

In [2]:
decamCCD = fits.open('../bricks_data/ccds-annotated-decam-dr9.fits')
mosaicCCD = fits.open('../bricks_data/ccds-annotated-mosaic-dr9.fits')
bassCCD = fits.open('../bricks_data/ccds-annotated-90prime-dr9.fits')
print(decamCCD[1].columns)


ColDefs(
    name = 'image_filename'; format = '120A'
    name = 'image_hdu'; format = 'I'
    name = 'camera'; format = '5A'
    name = 'expnum'; format = 'K'
    name = 'plver'; format = '8A'
    name = 'procdate'; format = '19A'
    name = 'plprocid'; format = '7A'
    name = 'ccdname'; format = '3A'
    name = 'object'; format = '35A'
    name = 'propid'; format = '10A'
    name = 'filter'; format = '1A'
    name = 'exptime'; format = 'E'
    name = 'mjd_obs'; format = 'D'
    name = 'airmass'; format = 'E'
    name = 'fwhm'; format = 'E'
    name = 'width'; format = 'I'
    name = 'height'; format = 'I'
    name = 'ra_bore'; format = 'D'
    name = 'dec_bore'; format = 'D'
    name = 'crpix1'; format = 'E'
    name = 'crpix2'; format = 'E'
    name = 'crval1'; format = 'D'
    name = 'crval2'; format = 'D'
    name = 'cd1_1'; format = 'E'
    name = 'cd1_2'; format = 'E'
    name = 'cd2_1'; format = 'E'
    name = 'cd2_2'; format = 'E'
    name = 'yshift'; format = 'L'
    name = 

In [3]:
dataDecam = decamCCD[1].data
dataMosaic = mosaicCCD[1].data
dataBass = bassCCD[1].data

In [4]:
ra0 = dataDecam.field('ra0')
dec0 = dataDecam.field('dec0')
ra1 = dataDecam.field('ra1')
dec1 = dataDecam.field('dec1')
ra2 = dataDecam.field('ra2')
dec2 = dataDecam.field('dec2')
ra3 = dataDecam.field('ra3')
dec3 = dataDecam.field('dec3')

In [16]:
import pandas as pd
df = pd.read_csv('../bricks_data/galaxy_catalogue_sample.csv')

ra = df["RA"].to_numpy(copy=True)
dec = df["DEC"].to_numpy(copy=True)

raDec = np.stack((ra,dec), axis=1)

raDec.shape

(379416, 2)

In [20]:
import matplotlib.path as mplPath


for no in range(len(ra0)):

    x0 = ra0[no]
    x1 = ra1[no]
    x2 = ra2[no]
    x3 = ra3[no]

    y0 = dec0[no]
    y1 = dec1[no]
    y2 = dec2[no]
    y3 = dec3[no]

    boundingBoxPath = mplPath.Path(np.array([[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]))

    no_objects_in_ccd = boundingBoxPath.contains_points(raDec).sum()
    if no_objects_in_ccd > 0:
        print(no, no_objects_in_ccd)

791 161
797 92
855 104
856 131
861 45
862 65
976 473
982 314
988 167
995 451
1002 502
1008 641
1013 717
1031 675
1037 409
1043 183
1050 390
1057 698
1063 1054
1261 78
1265 181
1269 585
1275 1014
1281 920
1287 365
1294 442
1301 348
1312 316
1317 442
1319 400
1332 143
1333 20
1338 142
1339 36
1438 13
1498 13
1512 145
1518 193
2267 36
2268 98
2273 171
2454 18
2455 100
2458 204
2497 15
2498 24
2503 93
2504 153
2509 99
2510 1
2515 37
2518 151
7285 22
7286 25
7292 127
7293 89
7299 59
7688 94
7689 30
7693 33
7694 142
7904 47
7905 48
7910 169
7919 89
7943 123
7944 6
7950 215
7951 21
7978 9
8003 122
8004 6
8010 215
8011 21
8038 9
8297 4
8303 225
8310 98
8357 5
8363 224
8370 98
8696 51
8699 84
8756 51
8759 86
8903 53
8910 137
8917 1
8941 8
8984 92
8989 53
8990 140
8994 7
9314 18
9315 48
9320 90
9321 115
9327 47
9455 60
9461 102
9462 93
9467 31
9468 35
9797 1
9803 31
10071 72
10072 18
10075 23
10076 195
10079 10
10812 135
10818 189
10936 832
10942 754
10955 98
10962 197
10973 107
10980 210
10981 

KeyboardInterrupt: 

In [28]:
def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))


In [ ]:
import healpy as hp

NSIDE = 256
NPIX = hp.nside2npix(NSIDE)

pixels_for_ccd = []

for no in range(len(ra0)):
    x_coord_polygon = np.array([ra0[no],ra1[no], ra2[no], ra3[no]])
    y_coord_polygon = np.array([dec0[no],dec1[no], dec2[no], dec3[no]])
    theta, phi = raDec2thetaPhi(x_coord_polygon, y_coord_polygon)

    ccd_vertices_3d = hp.ang2vec(theta=theta, phi=phi)
    pixels_for_ccd.append(hp.query_polygon(nside=NSIDE, vertices=ccd_vertices_3d, inclusive=True))
    if no % 5000 == 0:
        print(no)


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000


In [ ]:
expnumD = dataDecam.field('expnum')
expnumM = dataMosaic.field('expnum')
expnumB = dataBass.field('expnum')
expnum = np.concatenate((expnumB,expnumD, expnumM))
print(len(expnumB))